<a href="https://colab.research.google.com/github/QianFu520/food_sales_predictions/blob/main/Project_1_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#import libraries 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

In [39]:
df = pd.read_csv('/content/sales_predictions.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [40]:
#inspect data
print(df.info(), '\n')
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB
None 

Item_Identifier                 0
Item

I Can see that there are 1463 values missing in Item_weight, 2410 values missing in outlet_size 

In [41]:
#define X, y, train test split
X = df.drop(columns= 'Item_Outlet_Sales')
y = df["Item_Outlet_Sales"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
display(X_train.info())
X_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6392 entries, 4776 to 7270
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6392 non-null   object 
 1   Item_Weight                5285 non-null   float64
 2   Item_Fat_Content           6392 non-null   object 
 3   Item_Visibility            6392 non-null   float64
 4   Item_Type                  6392 non-null   object 
 5   Item_MRP                   6392 non-null   float64
 6   Outlet_Identifier          6392 non-null   object 
 7   Outlet_Establishment_Year  6392 non-null   int64  
 8   Outlet_Size                4580 non-null   object 
 9   Outlet_Location_Type       6392 non-null   object 
 10  Outlet_Type                6392 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 599.2+ KB


None

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,NCG06,16.350,Low Fat,0.029565,Household,256.4646,OUT018,2009,Medium,Tier 3,Supermarket Type2
7510,FDV57,15.250,Regular,0.000000,Snack Foods,179.7660,OUT018,2009,Medium,Tier 3,Supermarket Type2
5828,FDM27,12.350,Regular,0.158716,Meat,157.2946,OUT049,1999,Medium,Tier 1,Supermarket Type1
5327,FDG24,7.975,Low Fat,0.014628,Baking Goods,82.3250,OUT035,2004,Small,Tier 2,Supermarket Type1
4810,FDD05,19.350,Low Fat,0.016645,Frozen Foods,120.9098,OUT045,2002,NaN,Tier 2,Supermarket Type1


In [42]:
#Instantiate the Transformers
mean_imputer = SimpleImputer(strategy= 'mean')
most_frequent_imputer = SimpleImputer(strategy='most_frequent')
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()

In [43]:
#create pipeline for numeric data and categorical data
number_pipe = make_pipeline(mean_imputer, scaler)
ordinal_pipe = make_pipeline(most_frequent_imputer)
nominal_pipe = make_pipeline(ohe_encoder)

In [44]:
#Create Tuples to Pair Pipelines with Columns
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')


ord_cols = ['Outlet_Size']
number_tuple = (number_pipe, num_selector)
cat_tuple = (nominal_pipe, cat_selector)
ordinal_tuple = (ordinal_pipe, ord_cols)

In [45]:
#Instantiate the Pipeline
preprocessor = make_column_transformer(ordinal_tuple,
                                       cat_tuple,
                                       number_tuple,
                                       remainder='drop')


In [46]:
#ordinal encoder outlet size
size = {"Small": 0, "Medium": 1, "High": 2}
ordinal = df["Outlet_Size"].replace(size)
ordinal

0       1.0
1       1.0
2       1.0
3       NaN
4       2.0
       ... 
8518    2.0
8519    NaN
8520    0.0
8521    1.0
8522    0.0
Name: Outlet_Size, Length: 8523, dtype: float64

In [47]:
preprocessor.fit(X_train) #Fit the Pipeline on the Training Data
#Transform both the training data and the testing data
X_train_processed =preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)
X_train_processed

array([['Medium', 0.0, 0.0, ..., -0.7127750716052249, 1.8281092189993307,
        1.3278489341413338],
       ['Medium', 0.0, 0.0, ..., -1.291052247079188, 0.6033688805132759,
        1.3278489341413338],
       ['Medium', 0.0, 0.0, ..., 1.8133186433439548, 0.24454055715071762,
        0.13618724414043062],
       ...,
       ['Medium', 0.0, 0.0, ..., -0.9205271269932742, 1.5230267401668727,
        0.49368575114070157],
       ['Medium', 0.0, 0.0, ..., -0.22775520135060978,
        -0.38377707627087376, 1.089516596141153],
       ['Small', 0.0, 0.0, ..., -0.9586768265695343, -0.7383610459950399,
        -0.10214509385974999]], dtype=object)